# CatBoost pipeline

Attention à la version de numpy : 1.26.4 ( au lieu de 2.2.2) pour que ça fonctionne ici

In [1]:
import pandas as pd 

dataframe = pd.read_csv("SBAnational.csv", low_memory=False)

D'abord effectuer les conversions, avant de supprimer des colonnes

In [2]:
import conversion_functions as conv
import math

print (f"Before : {dataframe.shape[0]} lines")

explanable_X = pd.DataFrame(dataframe)

explanable_X["State"] = explanable_X["State"].apply(lambda x : conv.get_state_code(x))
explanable_X["NAICS"] = explanable_X["NAICS"].apply(lambda x : conv.get_NAICS_data(x))

explanable_X["ApprovalFY"] = explanable_X["ApprovalFY"].apply(lambda x : conv.get_ApprovalFY_data(x)) 
mean_dataframe = explanable_X[ explanable_X["ApprovalFY"] !=50]
mean_value = mean_dataframe["ApprovalFY"].mean()
explanable_X.loc[explanable_X['ApprovalFY'] == 50, 'ApprovalFY'] = math.ceil(mean_value)

explanable_X["NewExist"] = explanable_X["NewExist"].apply(lambda x : conv.get_NewExist_data(x)) 
explanable_X["FranchiseCode"] = explanable_X["FranchiseCode"].apply(lambda x : conv.get_FranchiseCode_data(x)) 

explanable_X = explanable_X.drop("UrbanRural", axis=1)

explanable_X["RevLineCr"] = explanable_X["RevLineCr"].apply(lambda x : conv.get_RevLineCr_data(x)) 
explanable_X["LowDoc"] = explanable_X["LowDoc"].apply(lambda x : conv.get_LowDoc_data(x)) 

explanable_X["GrAppv"] = explanable_X["GrAppv"].apply(lambda x : conv.get_GrAppv_value(x)) 
mean_dataframe = explanable_X[ explanable_X["GrAppv"] !=0]
mean_value = mean_dataframe["GrAppv"].mean()
explanable_X.loc[explanable_X['GrAppv'] == 0, 'ApprovalFY'] = math.ceil(mean_value)

explanable_X = explanable_X[ explanable_X["GrAppv"] !=0] 

explanable_X = explanable_X.drop("SBA_Appv", axis = 1)

explanable_X['MIS_Status'] = explanable_X.apply(lambda row: conv.predict_MIS_Status_data(row), axis=1 )
explanable_X["MIS_Status"] = explanable_X["MIS_Status"].apply(lambda x : conv.get_MIS_Status_data(x))

print (f"After : {explanable_X.shape[0]} lines")


Before : 899164 lines
After : 899164 lines


Ne supprimer les colonnes qu'après les conversions ( qui utilisent les colonnes supprimées )

In [3]:
explanable_X = explanable_X.drop('LoanNr_ChkDgt', axis=1)
explanable_X = explanable_X.drop('Name', axis=1)
explanable_X = explanable_X.drop('City', axis=1)
explanable_X = explanable_X.drop('Zip', axis=1)
explanable_X = explanable_X.drop('Bank', axis=1)
explanable_X = explanable_X.drop('BankState', axis=1)
explanable_X = explanable_X.drop('ApprovalDate', axis=1)
explanable_X = explanable_X.drop('ChgOffDate', axis=1) # (explicit end on simulation?) : included 
explanable_X = explanable_X.drop('DisbursementDate', axis=1)  
explanable_X = explanable_X.drop('DisbursementGross', axis=1)  
explanable_X = explanable_X.drop('BalanceGross', axis=1)  # 14 valeurs seulement différentes de zéro  ?
explanable_X = explanable_X.drop('ChgOffPrinGr', axis=1)

In [4]:
explanable_X["MIS_Status"].value_counts()

MIS_Status
P I F     741345
CHGOFF    157819
Name: count, dtype: int64

In [5]:
import numpy
import pandas as pd 

from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, RobustScaler, OrdinalEncoder, LabelEncoder, LabelBinarizer, OneHotEncoder, Binarizer, FunctionTransformer, PolynomialFeatures,MinMaxScaler

def identity_operation(X) :
    return X

preprocessor = make_pipeline(  
    make_column_transformer(
        # LoanNr_ChkDgt, Name , City : excluded

        (FunctionTransformer(identity_operation, validate=False), ["State"]), 
        # Zip , Bank , BankState : excluded

        (FunctionTransformer(identity_operation, validate=False), ["NAICS"]), #included (2 first chars )
        
        # ApprovalDate : excluded
        (StandardScaler(), ["ApprovalFY"]),
        (StandardScaler(), ["Term"]), 
        (StandardScaler(), ["NoEmp"]),
        (StandardScaler(), ["NewExist"]),
        (StandardScaler(), ["CreateJob"]),
        (StandardScaler(), ["RetainedJob"]),
        (Binarizer(), ["FranchiseCode"]),
        #Binarizer("UrbanRural", threshold=1.5),

        (FunctionTransformer(identity_operation, validate=False), ["RevLineCr"]),
        (Binarizer(), ["LowDoc"]),
        # SimpleImputer("ChgOffDate"), # explicit end on simulation
        # DisbursementDate, DisbursementGross , BalanceGross ( 14 valeurs seulement différentes de zéro  ?)
        # MIS_Status : Excluded because it is Y !!!
        # ChgOffPrinGr : excluded
        (StandardScaler(), ["GrAppv"]), 
        #(StandardScaler(),["SBA_Appv"]),  
        remainder='passthrough'))

In [6]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split
from catboost import CatBoostClassifier, Pool
from scipy.stats import uniform, randint

# CatBoost prend en charge les données catégorielles nativement
categorical_features = ['State','NAICS', 'RevLineCr']

y = explanable_X['MIS_Status']
X = explanable_X.drop('MIS_Status', axis=1)

# Split en train et test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Préparation des données pour CatBoost (en spécifiant les caractéristiques catégorielles)
# Convertir les données d'entraînement en un objet Pool pour indiquer les variables catégorielles (si nécessaire)
train_pool = Pool(data=X_train, label=y_train, cat_features=categorical_features)

test_pool = Pool(data=X_test, label=y_test, cat_features=categorical_features)


In [7]:
#durée estimée : 10 min (50 fits)

# Initialize CatBoostClassifier
cat_boost_model = CatBoostClassifier(learning_rate=0.1, iterations=100, depth=6, cat_features=categorical_features)

pipeline = Pipeline([
    ('preprocessor', preprocessor), 
    ('classifier', cat_boost_model)])

# Define the hyperparameters grid for RandomizedSearch
param_dist = {
    'iterations': randint(50, 200),       # Number of boosting iterations
    'depth': randint(4, 10),              # Depth of the trees
    'learning_rate': uniform(0.01, 0.3),  # Learning rate
    'l2_leaf_reg': uniform(1, 10),        # L2 regularization coefficient
    'border_count': randint(32, 255),     # Number of splits for categorical features
    'bagging_temperature': uniform(0, 1), # Controls the randomness in bagging
}

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(cat_boost_model, param_distributions=param_dist, 
                                   n_iter=10, scoring='roc_auc', 
                                   cv=5, verbose=1, n_jobs=-1)

# Fit the RandomizedSearchCV
random_search.fit(X_train, y_train)

# Print the best parameters and the best score
print("Meilleurs paramètres trouvés : ", random_search.best_params_)
print("Meilleur score AUC : ", random_search.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
0:	learn: 0.6184969	total: 436ms	remaining: 36.2s
0:	learn: 0.6186639	total: 593ms	remaining: 49.2s
0:	learn: 0.6184893	total: 578ms	remaining: 48s
0:	learn: 0.6185563	total: 675ms	remaining: 56s
0:	learn: 0.6184176	total: 744ms	remaining: 1m 1s
1:	learn: 0.5601770	total: 1.23s	remaining: 50.3s
2:	learn: 0.5144567	total: 1.57s	remaining: 42.5s
1:	learn: 0.5600395	total: 1.39s	remaining: 56.9s
1:	learn: 0.5601629	total: 1.42s	remaining: 58.4s
0:	learn: 0.6190924	total: 1.08s	remaining: 3m 20s
1:	learn: 0.5602804	total: 1.54s	remaining: 1m 3s
3:	learn: 0.4760314	total: 2.07s	remaining: 41.4s
1:	learn: 0.5601227	total: 1.7s	remaining: 1m 9s
0:	learn: 0.6201227	total: 1.45s	remaining: 4m 28s
2:	learn: 0.5144367	total: 1.98s	remaining: 53.5s
0:	learn: 0.5364519	total: 957ms	remaining: 2m 10s
0:	learn: 0.6196164	total: 1.03s	remaining: 3m 11s
2:	learn: 0.5146536	total: 2.15s	remaining: 58s
2:	learn: 0.5140587	total: 2.28s	remaining

In [8]:
#durée estimée : 40 min (200 fits), réelle : 

#Meilleur score AUC :  0.9771748465313973 avec one_hot_encoder
# param_dist = {
#     'iterations': [191],       # Number of boosting iterations
#     'depth': [7],              # Depth of the trees
#     'learning_rate': [0.2621537355139263],  # Learning rate
#     'l2_leaf_reg': [9.620670562880655],        # L2 regularization coefficient
#     'bagging_temperature': [0.09440670531484197], # Controls the randomness in bagging
# }

#Meilleur score AUC : 0.9773692063410346
param_dist = {
    'iterations': [173], 
    'depth': [8], 
    'learning_rate':[0.2628238092699549],
    'l2_leaf_reg': [4.642981495447009], 
    'bagging_temperature': [0.6728806036024608], 
    'border_count': [106]
}


# Initialize CatBoostClassifier
cat_boost_model = CatBoostClassifier(iterations = 150, learning_rate=0.2, depth=8, cat_features=categorical_features)

pipeline = Pipeline([
    ('preprocessor', preprocessor), 
    ('classifier', cat_boost_model)])

# Utilisation dans un gridsearch

best_params_random = random_search.best_params_

param_grid = {
    'iterations': [171, 172, 173, 174, 175],
    'depth': [7, 8, 9],
    'learning_rate': [0.25, 0.26, 0.27],
    'l2_leaf_reg': [4.642981495447009],
    'border_count': [106],
    'bagging_temperature': [0.6728806036024608]
}

# Créer le GridSearchCV avec le param_grid
grid_search = GridSearchCV(estimator=cat_boost_model, param_grid=param_grid, 
                           scoring='roc_auc', cv=5, n_jobs=-1, verbose=1)

# Ajuster GridSearchCV
grid_search.fit(X_train, y_train)

# Afficher les meilleurs paramètres et le meilleur score
print("Meilleurs paramètres trouvés par GridSearch : ", grid_search.best_params_)
print("Meilleur score AUC de GridSearch : ", grid_search.best_score_)


Fitting 5 folds for each of 45 candidates, totalling 225 fits
0:	learn: 0.5441383	total: 812ms	remaining: 2m 17s
0:	learn: 0.5442686	total: 1.09s	remaining: 3m 6s
0:	learn: 0.5450985	total: 799ms	remaining: 2m 15s
0:	learn: 0.5391733	total: 617ms	remaining: 1m 44s
1:	learn: 0.4534407	total: 1.73s	remaining: 2m 26s
0:	learn: 0.5390385	total: 1.3s	remaining: 3m 41s
0:	learn: 0.5447942	total: 1.1s	remaining: 3m 6s
1:	learn: 0.4537752	total: 1.71s	remaining: 2m 24s
0:	learn: 0.5396055	total: 1.21s	remaining: 3m 26s
1:	learn: 0.4476097	total: 1.62s	remaining: 2m 16s
1:	learn: 0.4535586	total: 2.33s	remaining: 3m 16s
0:	learn: 0.5446865	total: 1.64s	remaining: 4m 38s
1:	learn: 0.4470921	total: 2.38s	remaining: 3m 21s
0:	learn: 0.5400315	total: 999ms	remaining: 2m 49s
2:	learn: 0.3919935	total: 3.02s	remaining: 2m 49s
0:	learn: 0.5397178	total: 913ms	remaining: 2m 35s
0:	learn: 0.5341422	total: 1.1s	remaining: 3m 7s
1:	learn: 0.4541674	total: 2.49s	remaining: 3m 30s
2:	learn: 0.3941935	total:

KeyboardInterrupt: 

In [ ]:
#%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_auc_score
import MIS_Status_reference as mis_ref

test_score = random_search.score(X_test, y_test)

# Prédictions sur le jeu de test
y_pred = random_search.predict(X_test)

str_y_test = pd.DataFrame(y_test)
str_y_test["MIS_Status"]= str_y_test["MIS_Status"].apply(lambda x : mis_ref.convert_MIS_Status_to_string(x) )
# y_pred = y_pred.apply(lambda x : 1 if x=="P I F" else 0 )

# Évaluation du modèle
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(str_y_test, y_pred)
recall = recall_score(str_y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# Afficher les résultats
print(f"Accuracy : {accuracy * 100:.2f}%")
print(f"Precision : {accuracy * 100:.2f}%")
print(f"Recall : {accuracy * 100:.2f}%")
print("Matrice de confusion :")
print(conf_matrix)

# Prédire sur l'ensemble de test
y_pred_proba = random_search.predict_proba(X_test)[:, 1]  # Probabilités pour la classe positive

# Calculer l'AUC sur l'ensemble de test
auc = roc_auc_score(y_test, y_pred_proba)
print("AUC sur l'ensemble de test : ", auc)

# Calculer et afficher la matrice de confusion
conf_matrix = confusion_matrix(y_test, y_pred)
print("Matrice de confusion :")
print(conf_matrix)

# Affichage avec Seaborn (optionnel)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Classe 0', 'Classe 1'], yticklabels=['Classe 0', 'Classe 1'])
plt.xlabel('Prédictions')
plt.ylabel('Vraies étiquettes')
plt.title('Matrice de Confusion')
plt.show()

from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

# # Afficher la matrice de confusion
# print("Matrice de confusion :\n", conf_matrix)
# plt.figure(figsize=(6, 6))
# plot_confusion_matrix(random_search.best_estimator_, X_test, y_test, cmap=plt.cm.Blues)
# plt.title('Matrice de Confusion')
# plt.show()


In [ ]:
from sklearn.metrics import f1_score

# y_test : vraies étiquettes
# y_pred : prédictions du modèle
macro_f1 = f1_score(y_test, y_pred, average='macro')

print("Macro F1 Score : ", macro_f1)